# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Нам необходимо построить модель оттока клиентов. Набор данных о клиентах содержит следующие поля:

Признаки:
- `RowNumber` — индекс строки в данных
- `CustomerId` — уникальный идентификатор клиента
- `Surname` — фамилия
- `CreditScore` — кредитный рейтинг
- `Geography` — страна проживания
- `Gender` — пол
- `Age` — возраст
- `Tenure` — сколько лет человек является клиентом банка
- `Balance` — баланс на счёте
- `NumOfProducts` — количество продуктов банка, используемых клиентом
- `HasCrCard` — наличие кредитной карты
- `IsActiveMember` — активность клиента
- `EstimatedSalary` — предполагаемая зарплата

Целевой признак
- `Exited` — факт ухода клиента


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.utils import shuffle


In [2]:
data = pd.read_csv('Churn.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Исследуем данные каждого столбца. Посмотрим количество уникальных значений, сравним имеющиеся значения с типом данных, который присвоен при загрузке, определим категориальные параметры, которые необходимо преобразовать по методологии *One-Hot Encoding*. Также определим, какие параметры будут мешать построению моделей.

In [5]:
data['CustomerId'].nunique()

10000

Т.е. все объекты уникальные.

In [6]:
data['Geography'].nunique()

3

In [7]:
sorted(data['Tenure'].unique())

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, nan]

In [8]:
data['Tenure'].isna().sum()

909

Признак "сколько лет человек является клиентом банка" содержит только целочисленные значения. Еще есть 909 объектов с отсутствующей информацией о продолжительности сотрудничества с Клиентом. Данный показатель является одним из ключевых. Восстановить его по другим данным не представляется возможным. Возможны два варианта.
1. удалить объекты, где не известно время, в течение которого клиент сотрудничал с Банком
2. заполнить пропущенные значения случайными числами

Поскольку пропущенных значений почти 10% от всех данных, лучше сделать замену на случайные числа, чтобы не потерять остальные данные, которые хорошо бы учитывать при построении моделей.

In [9]:
np.random.seed()

In [10]:
data['Tenure'] = data['Tenure'].apply(lambda l: l if not np.isnan(l) else np.random.randint(10))

In [11]:
sorted(data['Tenure'].unique())

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [12]:
data['Tenure'] =  data['Tenure'].astype(int)

In [13]:
sorted(data['Tenure'].unique())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int32  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int32(1), int64(8), object(3)
memory usage: 1.0+ MB


In [15]:
data['HasCrCard'].unique()

array([1, 0], dtype=int64)

In [16]:
data['IsActiveMember'].unique()

array([1, 0], dtype=int64)

In [17]:
data['Exited'].unique()

array([1, 0], dtype=int64)

Для целей дальнейшего обучения первые три столбца набора данных лишние, это:
 - `RowNumber` — индекс строки в данных
 - `CustomerId` — уникальный идентификатор клиента
 - `Surname` — фамилия
 Чтобы эти параметры не мешали при обучении модели, удалим их в целевом наборе данных

In [18]:
data_ohe = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Параметры `Geography` и `Gender` являются категориальными. Поэтому для них создадим для каждого из этих признаков *One-Hot Encoding*, при этом будем обходить "дамми-ловушку".

In [19]:
data_ohe = pd.get_dummies(data_ohe, drop_first=True)

In [20]:
data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  int32  
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Geography_Germany  10000 non-null  uint8  
 10  Geography_Spain    10000 non-null  uint8  
 11  Gender_Male        10000 non-null  uint8  
dtypes: float64(2), int32(1), int64(6), uint8(3)
memory usage: 693.5 KB


### Spliting data

Целевым признаком является столбец `Exited` — факт ухода клиента. Остальные столбцы являются признаками.
Разделим исходный набор данных на обучающий набор, валидационный и тестовый в отношении 60%, 20% и 20%

In [21]:
target = data_ohe['Exited']

In [22]:
features = data_ohe.drop('Exited', axis=1)

In [23]:
features_train, features_test, target_train, target_test = \
                                 train_test_split(features, target, test_size=0.4, random_state=12345)

In [24]:
features_test, features_valid, target_test, target_valid = \
                                 train_test_split(features_test, target_test, test_size=0.5, random_state=12345)

### Data normalization

To avoid excessive influence of numerical variables on the modelling results, we'll normalize this data

In [25]:
scaler = StandardScaler()

In [26]:
numeric=['CreditScore','Age', 'Tenure', 'Balance', 'EstimatedSalary']

In [27]:
scaler.fit(features_train[numeric])

StandardScaler()

In [28]:
features_train[numeric] = scaler.transform(features_train[numeric])

In [29]:
features_test[numeric] = scaler.transform(features_test[numeric])

In [30]:
features_valid[numeric] = scaler.transform(features_valid[numeric])

In [31]:
features_train[numeric].head()

,CreditScore,Age,Tenure,Balance,EstimatedSalary
7479,-0.886751,-0.373192,1.042635,1.232271,-0.187705
3411,0.608663,-0.183385,1.042635,0.600563,-0.333945
6027,2.052152,0.480939,-0.689029,1.027098,1.503095
1247,-1.457915,-1.417129,0.349969,-1.233163,-1.071061
3716,0.130961,-1.132419,-1.035362,1.140475,1.524268


In [32]:
features_test[numeric].head()

,CreditScore,Age,Tenure,Balance,EstimatedSalary
8532,-0.699824,-0.373192,-1.035362,-1.233163,-0.015173
5799,-0.284431,0.575842,-0.689029,-1.233163,1.471724
5511,0.151731,-0.657902,-1.728028,0.438711,-1.367107
7365,-0.876366,-0.278288,1.735301,1.239884,-0.786517
7367,-0.481743,0.291132,1.735301,-1.233163,1.358533


In [33]:
features_valid[numeric].head()

,CreditScore,Age,Tenure,Balance,EstimatedSalary
7041,-2.226392,-0.088482,-1.035362,-1.233163,0.647083
5709,-0.087120,0.006422,1.388968,-1.233163,-1.658410
7117,-0.917905,-0.752805,0.003636,0.722307,-1.369334
7775,-0.253277,0.101325,1.735301,-1.233163,0.075086
8735,0.785204,-0.847708,1.735301,0.615625,-1.070919


Были подготовлены данные для дальнейшего исследования на различных моделях. В нашем распоряжении есть набор данных для обучения, для валидации обученных моделей и для финального тестирования полученной модели.

## Исследование задачи

В данном разделе исследуем модели на имеющихся данных

### Логистическая регрессия

In [34]:
model = LogisticRegression(solver='liblinear' ,random_state=12345)

In [35]:
model.fit(features_train, target_train)

LogisticRegression(random_state=12345, solver='liblinear')

In [36]:
predict_valid = model.predict(features_valid)

In [37]:
f1_score(predict_valid, target_valid)

0.2777777777777778

Посомтрим значение параметра *AUC_ROC*

In [38]:
probabilities_valid = model.predict_proba(features_valid)

In [39]:
probabilities_one_valid = probabilities_valid[:, 1]

In [40]:
roc_auc_score(target_valid, probabilities_one_valid)

0.7390907414653014

### Дерево решений

Для дерева решений на валидационной выборке выберем глубину дерева

In [41]:
f1_tree_row_best = 0
depth_tree_row_optimum = 0

for depth in range(1,9):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    f1 = f1_score(predict_valid, target_valid)
    if f1_tree_row_best < f1:
        f1_tree_row_best = f1
        depth_tree_row_optimum = depth

In [42]:
f1_tree_row_best

0.5337423312883436

In [43]:
depth_tree_row_optimum

6

Посмотрим чему равен AUC_ROC

In [44]:
model = DecisionTreeClassifier(random_state=12345, max_depth=depth_tree_row_optimum)
model.fit(features_train, target_train)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8394526219847661

Значения F1 и AUC_ROC значительно лучше, чем для логистической регрессии. 
Теперь посмотрим какой результат даст случайный лес.

### Случайный лес

Для данной модели сперва подберем оптимальные значения параметров для максимальной глубины одного дерева и для количества деревьев

In [45]:
best_forest_row_est = 0
best_forest_row_depth = 0
best_forest_row_f1 = 0

for est in range(10, 511, 100):
    for depth in range (1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predict_valid = model.predict(features_valid)
        f1 = f1_score(predict_valid, target_valid)
        if best_forest_row_f1 < f1:
            best_forest_row_f1 = f1
            best_forest_row_est = est
            best_forest_row_depth = depth          

In [46]:
best_forest_row_est

510

In [47]:
best_forest_row_depth

10

In [48]:
best_forest_row_f1

0.5195618153364633

Для найденного оптимума параметров леса качество модели становится немного лучше чем для одиночного решающего дерева.

Для "сырого" набора данных степень точности моделей получается не очень высоким. Посмотрим как может измениться точность модели при увеличении количества деревьев

In [49]:
model = RandomForestClassifier(random_state=12345, n_estimators=1000, max_depth=best_forest_row_depth)
model.fit(features_train, target_train)

predict_valid = model.predict(features_valid)
f1_score(predict_valid, target_valid)

0.51875

Точность модели ухудшается. Посмотрим показатель AUC_ROC для оптимальных параметров *количество деревьев* и *максимальная глубина дерева*.

In [50]:
model = RandomForestClassifier(random_state=12345, n_estimators=best_forest_row_est, max_depth=best_forest_row_depth)
model.fit(features_train, target_train)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8600763636854248

 Теперь исследуем исходный набор на предмет наличия дисбалансов

## Борьба с дисбалансом

### Предварительная оценка исходных данных

In [51]:
sum((target_train == 1))

1196

In [52]:
sum((target_train == 0))

4804

Таким образом количество положительных классов примерно в 4 раза меньше отрицательных. 

Для дальнейшего исследования проведем сперва увеличение положительных ответов а потом уменьшение отрицательных и посмотрим, в каком случае модель покажет более качественные результаты

### Увеличение выборки

#### Подготовка данных

Увеличим количество положительных ответов в 4 раза, перемешаем все объекты и после этого разделим исходные данные на обучающую, валидационную и тестовую выборки

In [53]:
features_train_zeros = features_train[target_train == 0]

In [54]:
features_train_ones = features_train[target_train == 1]

In [55]:
target_train_zeros = target_train[target_train == 0]

In [56]:
target_train_ones = target_train[target_train == 1]

In [57]:
features_train_upsampled = pd.concat([features_train_zeros] + [features_train_ones] * 4)

In [58]:
target_train_upsampled = pd.concat([target_train_zeros] + [target_train_ones] * 4)

In [59]:
features_train_upsampled, target_train_upsampled = shuffle(features_train_upsampled, target_train_upsampled, random_state=12345)

In [60]:
features_train_upsampled[target_train_upsampled==1].shape

(4784, 11)

In [61]:
features_train_upsampled[target_train_upsampled==0].shape

(4804, 11)

Выборка готова. Проверим наши модели на данном наборе данных.

#### Логистическая регрессия

In [62]:
model = LogisticRegression(solver='liblinear' ,random_state=12345)

In [63]:
model.fit(features_train_upsampled, target_train_upsampled)

LogisticRegression(random_state=12345, solver='liblinear')

In [64]:
predict_upsampled_valid = model.predict(features_valid)

In [65]:
f1_score(predict_upsampled_valid, target_valid)

0.48228176318063953

Точность модели *Логистическая регрессия* немного увеличилась. Однако в целом точность модели остается низкой. 

In [66]:
probabilities_upsampled_valid = model.predict_proba(features_valid)
probabilities_upsampled_one_valid = probabilities_upsampled_valid[:, 1]
roc_auc_score(target_valid, probabilities_upsampled_one_valid)

0.7418820485375619

Показатель AUC_ROC практически не изменился.

#### Дерево решений

In [67]:
f1_tree_upsampled_best = 0
depth_tree_upsampled_optimum = 0

for depth in range(1,9):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train_upsampled, target_train_upsampled)
    predict_upsampled_valid = model.predict(features_valid)
    f1 = f1_score(predict_upsampled_valid, target_valid)
    if f1_tree_upsampled_best < f1:
        f1_tree_upsampled_best = f1
        depth_tree_upsampled_optimum = depth

In [68]:
f1_tree_upsampled_best

0.5809128630705395

In [69]:
depth_tree_upsampled_optimum

5

Значение меры F1 увеличилось незначительно по сравнению с сырыми данными. при этом оптимальная глубина дерева уменьшилась с 7 до 5.

Посмотрим показатель *AUC_ROC*

In [70]:
model = DecisionTreeClassifier(random_state=12345, max_depth=depth_tree_upsampled_optimum)
model.fit(features_train_upsampled, target_train_upsampled)

probabilities_upsampled_valid = model.predict_proba(features_valid)
probabilities_upsampled_one_valid = probabilities_upsampled_valid[:, 1]
roc_auc_score(target_valid, probabilities_upsampled_one_valid)

0.8355347481752318

А значение AUC_ROC немного увеличилось, что говорит, что качество модели *Решающее дерево* немного улучшилось

#### Случайный лес

In [71]:
best_forest_upsampled_est = 0
best_forest_upsampled_depth = 0
best_forest_upsampled_f1 = 0

for est in range(10, 511, 100):
    for depth in range (1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train_upsampled, target_train_upsampled)
        predict_upsampled_valid = model.predict(features_valid)
        f1 = f1_score(predict_upsampled_valid, target_valid)
        if best_forest_upsampled_f1 < f1:
            best_forest_upsampled_f1 = f1
            best_forest_upsampled_est = est
            best_forest_upsampled_depth = depth

In [72]:
best_forest_upsampled_f1

0.617283950617284

In [73]:
best_forest_upsampled_est

210

In [74]:
best_forest_upsampled_depth

9

Для модели *Случайный лес* мера F1 превысила требуемое нам для точности значение. При этом оптимальное количество деревьев в лесу увеличилось почти в 4 раза, при том что глубина дерева не изменилась.

Посмотрим показатель *AUC_ROC*

In [75]:
model = RandomForestClassifier(random_state=12345, \
                               n_estimators=best_forest_upsampled_est, max_depth=best_forest_upsampled_depth)
model.fit(features_train_upsampled, target_train_upsampled)

probabilities_upsampled_valid = model.predict_proba(features_valid)
probabilities_upsampled_one_valid = probabilities_upsampled_valid[:, 1]
roc_auc_score(target_valid, probabilities_upsampled_one_valid)

0.8577752593052315

При этом значение *AUC_ROC* немного ухудшилось.

Посмотрим как изменится точность моделей при уменьшении выборки.

### Уменьшение выборки

#### Подготовка данных

Поскольку отрицательных данных у нас в 4 раза больше положительных, то коэффициент уменьшения будет принимать значение 0.26

In [76]:
features_train_downsampled = pd.concat([features_train_zeros.sample(frac=0.26, random_state=12345)] + [features_train_ones])
target_train_downsampled = pd.concat([target_train_zeros.sample(frac=0.26, random_state=12345)] + [target_train_ones])
features_train_downsampled, target_train_downsampled = shuffle(features_train_downsampled, target_train_downsampled, random_state=12345)

In [77]:
features_train_downsampled[target_train_downsampled==1].shape

(1196, 11)

In [78]:
features_train_downsampled[target_train_downsampled==0].shape

(1249, 11)

Данные подготовлены. Перейдем к проверке моделей

#### Логистическая регрессия

In [79]:
model = LogisticRegression(solver='liblinear' ,random_state=12345)

In [80]:
model.fit(features_train_downsampled, target_train_downsampled)

LogisticRegression(random_state=12345, solver='liblinear')

In [81]:
predict_downsampled_valid = model.predict(features_valid)

In [82]:
f1_score(predict_downsampled_valid, target_valid)

0.48336252189141865

In [83]:
probabilities_downsampled_valid = model.predict_proba(features_valid)
probabilities_downsampled_one_valid = probabilities_downsampled_valid[:, 1]
roc_auc_score(target_valid, probabilities_downsampled_one_valid)

0.7418265821779092

Для логистической регрессии параметры точности практически не изменились по сравнению с увеличенной выборкой.

#### Дерево решений

In [84]:
f1_tree_downsampled_best = 0
depth_tree_downsampled_optimum = 0

for depth in range(1,9):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train_downsampled, target_train_downsampled)
    predict_downsampled_valid = model.predict(features_valid)
    f1 = f1_score(predict_downsampled_valid, target_valid)
    if f1_tree_downsampled_best < f1:
        f1_tree_downsampled_best = f1
        depth_tree_downsampled_optimum = depth

In [85]:
f1_tree_downsampled_best

0.5926640926640927

In [86]:
depth_tree_downsampled_optimum

5

In [87]:
model = DecisionTreeClassifier(random_state=12345, max_depth=depth_tree_downsampled_optimum)
model.fit(features_train_downsampled, target_train_downsampled)

probabilities_downsampled_valid = model.predict_proba(features_valid)
probabilities_downsampled_one_valid = probabilities_downsampled_valid[:, 1]
roc_auc_score(target_valid, probabilities_downsampled_one_valid)

0.8321970225058501

Для модели *Дерево решений* параметры точности модели уменьшились по сравнению с увличенным набором данных.

#### Случайный лес

In [88]:
best_forest_downsampled_est = 0
best_forest_downsampled_depth = 0
best_forest_downsampled_f1 = 0

for est in range(10, 511, 100):
    for depth in range (1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train_downsampled, target_train_downsampled)
        predict_downsampled_valid = model.predict(features_valid)
        f1 = f1_score(predict_downsampled_valid, target_valid)
        if best_forest_downsampled_f1 < f1:
            best_forest_downsampled_f1 = f1
            best_forest_downsampled_est = est
            best_forest_downsampled_depth = depth

In [89]:
best_forest_downsampled_f1

0.6025267249757046

In [90]:
best_forest_downsampled_est

210

In [91]:
best_forest_downsampled_depth

8

In [92]:
model = RandomForestClassifier(random_state=12345, \
                               n_estimators=best_forest_downsampled_est, max_depth=best_forest_downsampled_depth)
model.fit(features_train_downsampled, target_train_downsampled)

probabilities_downsampled_valid = model.predict_proba(features_valid)
probabilities_downsampled_one_valid = probabilities_downsampled_valid[:, 1]
roc_auc_score(target_valid, probabilities_downsampled_one_valid)

0.8520412369897659

Модель *Случайный лес* на уменьшеной выборке показывает точность лучше, чем модель *Дерево решений*, но все равно это хуже, чем эта же модель показывает на увеличенной выборке.

## Тестирование модели

Самые лучшие результаты показала модель *Случайный лес* на увеличенной выборке. Проверим эту модель на тестовых данных.

In [93]:
model = momodel = RandomForestClassifier(random_state=12345, \
                               n_estimators=best_forest_upsampled_est, max_depth=best_forest_upsampled_depth)
model.fit(features_train_upsampled, target_train_upsampled)

RandomForestClassifier(max_depth=9, n_estimators=210, random_state=12345)

In [94]:
predict_upsampled_test = model.predict(features_test)

In [95]:
f1_score(predict_upsampled_test, target_test)

0.6210526315789473

In [96]:
probabilities_upsampled_test = model.predict_proba(features_test)
probabilities_upsampled_one_test = probabilities_upsampled_test[:, 1]
roc_auc_score(target_test, probabilities_upsampled_one_test)

0.8546885113023911

## Общий вывод

Таким образом мы можем видеть, что наилучшие ркзультаты для данного набора данных показывает модель *Случайный лес* на увеличенном наборе данных.

Значение меры **F1** равно **0.621**
Показатель **AUC_ROC** равен **0.8546**

Надо отметить, что показатель *AUC_ROC* для разных моделей и разных наборов данных меняется не так значительно как мера *F1*

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*

https://github.com/konbag/Supervised-learning